In [1]:
unprocessed_folder = "/home/gayan/Desktop/CMLARE/Data/Un Processed/20180318"

In [2]:
processed_folder = "/home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18"

In [12]:
import os
from shutil import copy
import csv
import geopy.distance

In [13]:
link_data = []
mapped_links = []
test_file = processed_folder+"/PM_IG30028_15_201803180000_01.csv"

mapped_links_file="mapped_links.csv"
mapped_links_field_names = ["cell_tower_id1","cell_tower_id2","source1_name","source2_name", "district1","district2",
                            "latitude1","latitude2","longitude1","longitude2","nearest_precip_station","distance",
                            "nearest_precip_lat","nearest_precip_long","id_1","id_2"]

link_data_field_names = ["DeviceID","DeviceName","ResourceID","ResourceName","CollectionTime","RSL_MAX","RSL_MIN","RSL_AVG","RSL_CUR","TSL_MAX","TSL_MIN","TSL_AVG","TSL_CUR"]

processed_data_all = []
processed_file = "/home/gayan/Desktop/CMLARE/Data/Processed/processed_file.csv"
processed_file_field_names = ["ID","Pmax","Pmin","XStart","YStart","XEnd","YEnd","DateTime","PathLength","Frequency"]

    #copy the usable files to the processed directory

In [72]:
directory = os.listdir(unprocessed_folder)
correct_files = [],""
for file in directory:
    splitted_name = file.split("_")[1]      
    if(splitted_name.endswith("28")):
        correct_files.append(file)
for file in correct_files:
    copy(src=unprocessed_folder+"/"+file,dst=processed_folder)

In [14]:
def start_mapping(files):
    global link_data,mapped_links_file,mapped_links_field_names,mapped_links,link_data_field_names,processed_data_all
    with open(mapped_links_file,"r") as mapped_links_csv:
        mapped_links = csv.DictReader(mapped_links_csv, fieldnames=mapped_links_field_names)
        next(mapped_links)                                                                                     #Skipping the Header
        mapped_links = list(mapped_links)
        print(" Reading Cell towers locations successfull")
        for file in files:
            with open(file,"r") as file_csv:
                print("processing file : "+file)
                link_data = csv.DictReader(file_csv,fieldnames=link_data_field_names)
                next(link_data)
                next(link_data)
                link_data = list(link_data)
                for link in link_data:
                    link_resource_name = link["ResourceName"]
                    processed_data = {}
                    processed_data["Pmin"] = link["RSL_MIN"]
                    processed_data["Pmax"] = link["RSL_MAX"]
                    processed_data["DateTime"] = link["CollectionTime"].replace("-","").replace(" ","").replace(":","")[:-2]
                    for mapped_link in mapped_links:
                        mapped_link_resource_name1 = mapped_link["source1_name"]
                        mapped_link_resource_name2 = mapped_link["source2_name"]
                        if(link_resource_name == mapped_link_resource_name1):
                            processed_data["XStart"] = mapped_link["latitude2"]
                            processed_data["YStart"] = mapped_link["longitude2"]
                            processed_data["XEnd"] = mapped_link["latitude1"]
                            processed_data["YEnd"] = mapped_link["longitude1"]
                            processed_data["ID"] = mapped_link["id_1"]
                            processed_data["Frequency"] = "35"
                            start_cord = (processed_data["XStart"],processed_data["YStart"])
                            end_cord = (processed_data["XEnd"],processed_data["YEnd"])
                            processed_data["PathLength"] = geopy.distance.vincenty(start_cord, end_cord).km
                            break
                        elif(link_resource_name == mapped_link_resource_name2):
                            processed_data["XStart"] = mapped_link["latitude1"]
                            processed_data["YStart"] = mapped_link["longitude1"]
                            processed_data["XEnd"] = mapped_link["latitude2"]
                            processed_data["YEnd"] = mapped_link["longitude2"]
                            processed_data["ID"] = mapped_link["id_2"]
                            processed_data["Frequency"] = "35"
                            start_cord = (processed_data["XStart"],processed_data["YStart"])
                            end_cord = (processed_data["XEnd"],processed_data["YEnd"])
                            processed_data["PathLength"] = geopy.distance.vincenty(start_cord, end_cord).km
                            break
                    if(("XStart" in processed_data) and ("YStart" in processed_data) and ("XEnd" in processed_data) and ("YEnd" in processed_data)):
                        processed_data_all.append(processed_data)
                print("processing file : "+file+" successfull")

                    
        with open(processed_file,"w") as processed_file_csv:
            dictionary_writer = csv.DictWriter(processed_file_csv,processed_file_field_names)
            dictionary_writer.writeheader()
            dictionary_writer.writerows(processed_data_all)
                    
def process_all():
    directory = os.listdir(processed_folder)
    directory = list(directory)
    correct_file_paths = []
    for file in directory:
#         print(file)
        correct_file_paths.append(str(processed_folder)+"/"+file)
#     print(directory)
    start_mapping(correct_file_paths)
    print("process Successfull")

process_all()

 Reading Cell towers locations successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180015_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180015_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180545_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180545_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180100_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180100_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181515_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181515_01.csv successfu

processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180330_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180500_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180500_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181445_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181445_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181830_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181830_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180045_01.csv
processing file : /home/gayan/Desktop/CM

processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181045_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180400_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803180400_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181415_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181415_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181345_01.csv
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181345_01.csv successfull
processing file : /home/gayan/Desktop/CMLARE/Data/Processed/selected/2018-03-18/PM_IG30028_15_201803181745_01.csv
processing file : /home/gayan/Desktop/CM